In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Live Orders with Holdings in LUSID

Attributes
----------
portfolio
recipe
quotes
instruments
holdings
orders
"""

toggle_code("Toggle Docstring")

# Incorporating live orders into your holdings view

In this example we demonstrate how to give front office users a view of intraday trading activity on top of their middle office IBOR. We set up a `Portfolio` with multiple strategies and generate `Orders` and `Allocations.` We can show updated positions that account for these live, partially allocated orders.

To illustrate, below we will trade <i>Amazon, Inc.</i>. We'll create a set of limit, stop, and market orders, and show an updated view of the portfolio intraday.


## Imports

In [2]:
from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os
from datetime import date,timedelta,datetime

import lusid
import lusid.models as lm
import lusid.api as la

from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

# Set DataFrame display formats
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 500)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.6.11310.0


Define scope, portfolio and some other variables used in the example:

In [3]:
orders_scope = "holdings_with_live_orders"
orders_portfolio = "us_long"
strategy_shk = "strategy"
date_today=date.today
pf_created_date = "2020-01-01T00:00:00+00:00"

recipe_scope = orders_scope
recipe_code = "holdings_with_orders"

instruments_scope = orders_scope


## Create a Portfolio

In [4]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code, "portfolio_name": name},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {
            "created": f"${pf_created_date}"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
    )

    succ, failed = format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])) 

In [5]:
create_portfolio(orders_scope, orders_portfolio, "Holdings + Live")

,success,failed
0,1,0


## Load an Instrument Master

In [6]:
instr_df = pd.read_csv("data/live_orders_instruments.csv")
display(instr_df)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Anglian_Water_40LV,imd_13579246,USD,XS0089553282,NaN,GB,united_kingdom,NaN,fixed_income,bond,6.625
2,ICE_Silver_5000oz_Dec21,imd_12457801,USD,NaN,NaN,GB,united_kingdom,ZIZ21,commodity,future,NaN


In [7]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}

In [8]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=instruments_scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
    instrument_scope=instruments_scope
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,3,0,0


## Create a recipe

In [9]:
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

configuration_recipe = lm.ConfigurationRecipe(
    recipe_scope,
    recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=orders_scope,
                quote_type="Price",
                field="mid",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=orders_scope,
        )
    ),
    pricing=lm.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

## Set initial holdings in the parent portfolio

In [10]:
hldgs_df = pd.read_csv("data/initial_amazon_holdings.csv")
display(hldgs_df)

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,165,825000,USD


In [11]:
holdings_mapping = {
    "required":{
        "code": f"${orders_portfolio}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity"
    },
    "identifier_mapping": {
        "ClientInternal": "client_internal",
    },
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost"
    }
}

In [12]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=orders_scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    file_type="holdings"
)
succ, failed = format_holdings_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


### Upsert Quotes
Load some sample data for our `Instruments.` We're uploading a `Quote` for today, as market orders are converted to holdings by assuming that they're filled at the instant we make the query and take the latest market price for it.

In [13]:
quotes_df = pd.read_excel("data/quotes.xlsx")
quotes_df['date']=date.today()
quotes_df.head()

,date,ticker,name,figi,Sector,open_price,close_price,client_internal
0,2023-05-16,AMZN,Amazon Nasdaq,BBG000BVPXP1,Equity Investment Instruments,111.00,111.5,imd_34634534
1,2023-05-16,NaN,Anglian Water 40LV,NaN,Fixed Income,110.93,112.2,imd_13579246
2,2023-05-16,ZIZ21,ICE Silver 5000oz Dec21,NaN,Commodities,24.30,25.4,imd_12457801


In [14]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$ClientInternal",
    "quote_id.effective_at": "date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "client_internal",
    "metric_value.unit": "$USD",
}

In [15]:
quotes_mapping["quote_id.quote_series_id.field"] ="$mid"
quotes_mapping["metric_value.value"] = "close_price"

result = load_from_data_frame(
    api_factory = api_factory,
    scope=orders_scope,
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)
succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

,success,failed,errors
0,3,0,0


## Check the initial portfolio holdings
We see just a single, material holding in the instrument.

In [16]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=orders_scope,
                    code=orders_portfolio,property_keys=["Instrument/default/Name"],
                    recipe_id_scope=recipe_scope,recipe_id_code=recipe_code)

response_df=lusid_response_to_data_frame(executed_holdings, rename_properties=True)
response_df[["instrument_uid", "holding_type", "units", "cost.amount", "cost.currency"]].style.format({"units":"{:20,.0f}","cost.amount": "{:20,.2f}"})

,instrument_uid,holding_type,units,cost.amount,cost.currency
0,LUID_00003DFK,P,"5,000","825,000.00",USD


## Post orders into LUSID

In this section, we post some [orders](https://support.finbourne.com/how-does-lusid-support-the-trade-lifecycle) into LUSID. We have a number of orders for Amazon stock to demonstrate the processing of market, limit, and stop orders. In addtion we have an order for a corporate bond.

In [17]:
orders_df = pd.read_csv('data/live_orders.csv')
orders_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side,type,state,limit_price,limit_currency,stop_price,stop_currency
0,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,1000,100.0,USD,ORD001,sell,limit,new,100.50,USD,NaN,NaN
1,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,1000,95.0,USD,ORD002,buy,limit,new,96.25,USD,NaN,NaN
2,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,3000,NaN,USD,ORD003,buy,stop,new,NaN,NaN,94.0,USD
3,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,1000,NaN,USD,ORD004,buy,market,new,NaN,NaN,NaN,NaN
4,us_long,Anglian_Water_40LV,imd_13579246,XS0089553282,NaN,500,NaN,USD,ORD005,buy,market,new,NaN,NaN,NaN,NaN
5,us_long,ICE_Silver_5000oz_Dec21,imd_12457801,NaN,NaN,5000,NaN,USD,ORD006,buy,market,new,NaN,NaN,NaN,NaN


In [18]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)
responses = []

for index, order in orders_df.iterrows():
    
    request = lm.OrderRequest(
            id=lm.ResourceId(
                scope=orders_scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': order['client_internal']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=orders_scope,
                code=orders_portfolio
            ),
            state=order['state'],
            type=order['type'],
            price=lm.CurrencyAndAmount(
                        amount=0 if pd.isna(order['price']) else order['price'],
                        currency=order['currency']
            ),

            limit_price=lm.CurrencyAndAmount(
                        amount=order['limit_price'],
                        currency=order['limit_currency']
            )            
            if not pd.isna(order['limit_price']) and not pd.isna(order['limit_currency']) else None,
            
            stop_price=lm.CurrencyAndAmount(
                        amount=order['stop_price'],
                        currency=order['stop_currency']
            )
            if not pd.isna(order['stop_price']) and not pd.isna(order['stop_currency']) else None
    )
    
    request=lm.OrderSetRequest(
        order_requests=[request]           
    )

    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=request
    )
    
    responses.append(response.values[0])

attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/ClientInternal'],
             o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount,o.price.currency,
             o.limit_price.amount if o.limit_price is not None else "N/A",
             o.limit_price.currency if o.limit_price is not None else "N/A") for o in responses]

pd.DataFrame(attributes, columns=['order_id','client_internal','lusid_instrument_id','side','type',
                                  'state','quantity','price','currency','lim px',
                                  'lim ccy']).style.format({"quantity":"{:20,.0f}","price": "{:20,.2f}"})

,order_id,client_internal,lusid_instrument_id,side,type,state,quantity,price,currency,lim px,lim ccy
0,ORD001,imd_34634534,LUID_00003DFK,sell,limit,new,"1,000",100.00,USD,100.500000,USD
1,ORD002,imd_34634534,LUID_00003DFK,buy,limit,new,"1,000",95.00,USD,96.250000,USD
2,ORD003,imd_34634534,LUID_00003DFK,buy,stop,new,"3,000",0.00,USD,N/A,N/A
3,ORD004,imd_34634534,LUID_00003DFK,buy,market,new,"1,000",0.00,USD,N/A,N/A
4,ORD005,imd_13579246,LUID_00003DFM,buy,market,new,500,0.00,USD,N/A,N/A
5,ORD006,imd_12457801,LUID_00003DFL,buy,market,new,"5,000",0.00,USD,N/A,N/A


### Post allocations into LUSID
In this section, we post some allocations into LUSID. These allocations represent full or partial allocations against a subset of the originating orders.

In [19]:
allocations_df = pd.read_csv('data/amazon_allocations.csv')
allocations_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,allocation_id,originating_order,state,side,type
0,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,777,102.7,USD,ALLOC001-for-ORD004,ORD004,partial allocation,buy,market


In [20]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)
responses = []

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    request = lm.AllocationRequest(
            id=lm.ResourceId(
                scope=orders_scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id = lm.ResourceId(
                scope=orders_scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            side=allocation['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': allocation['client_internal']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=orders_scope,
                code=orders_portfolio
            ),
            state=allocation['state'],
            type=allocation['type'],
            price=lm.CurrencyAndAmount(
                        amount=allocation['price'],
                        currency=allocation['currency']))
    
    request=lm.AllocationSetRequest(
        allocation_requests=[request]           
    )

    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        allocation_set_request=request
    )
    
    responses.append(response.values[0])

attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/ClientInternal'],o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount) for o in responses]
pd.DataFrame(attributes, columns=['allocation_id','client_internal','lusid_instrument_id','side','type','state','quantity','price']).style.format({"quantity":"{:20,.0f}","price": "${:20,.2f}"})

,allocation_id,client_internal,lusid_instrument_id,side,type,state,quantity,price
0,ALLOC001-for-ORD004,imd_34634534,LUID_00003DFK,buy,market,partial allocation,777,$ 102.70


## Check the portfolio holdings with orders
Finally we can ask LUSID to give us a live holdings view that includes contributions from outstanding orders of market, limit, or stop type.

We'll see Order (O) and OrderContra (OC) holdings for the security and cash commitments of each outstanding order. Partially and fully-allocated orders will also have Allocation (L) and AllocationContra (LC) holdings to represent allocations against them, and have their quantities suitably reduced.

In [21]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(
    scope=orders_scope, 
    code=orders_portfolio, 
    property_keys=["Instrument/default/Name"], 
    recipe_id_scope=recipe_scope, 
    recipe_id_code=recipe_code)

response_df = lusid_response_to_data_frame(executed_holdings, rename_properties=True)

response_df.rename(columns = {'transaction.transaction_price.price':'price','Name(default-Properties)':'instrument','cost.amount':'cost','cost.currency':'ccy','cost_portfolio_ccy.amount':'pfolio_cost'}, inplace = True)
response_df['transaction.transaction_id'] = response_df['transaction.transaction_id'].fillna('')
grouped=response_df.groupby('transaction.transaction_id')

for name,group in grouped:
    display(group[["instrument", "holding_type", "units", "cost", "ccy", "pfolio_cost",  "price"]].style.format({"price":"{:20,.2f}","units":"{:20,.0f}","cost": "{:20,.2f}","pfolio_cost": "{:20,.2f}"}).set_caption(name))


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
0,Amazon_Nasdaq_AMZN,P,"5,000","825,000.00",USD,"825,000.00",nan


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
13,Amazon_Nasdaq_AMZN,L,777,"79,797.90",USD,"79,797.90",102.70
14,USD,LC,"-79,798","-79,797.90",USD,"-79,797.90",102.70


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
11,Amazon_Nasdaq_AMZN,O,"-1,000","-100,500.00",USD,"-100,500.00",100.50
12,USD,OC,"100,500","100,500.00",USD,"100,500.00",100.50


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
9,Amazon_Nasdaq_AMZN,O,"1,000","96,250.00",USD,"96,250.00",96.25
10,USD,OC,"-96,250","-96,250.00",USD,"-96,250.00",96.25


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
7,Amazon_Nasdaq_AMZN,O,"3,000","282,000.00",USD,"282,000.00",94.00
8,USD,OC,"-282,000","-282,000.00",USD,"-282,000.00",94.00


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
5,Amazon_Nasdaq_AMZN,O,223,"24,864.50",USD,"24,864.50",111.50
6,USD,OC,"-24,864","-24,864.50",USD,"-24,864.50",111.50


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
1,Anglian_Water_40LV,O,500,"56,100.00",USD,"56,100.00",112.20
2,USD,OC,"-56,100","-56,100.00",USD,"-56,100.00",112.20


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
3,ICE_Silver_5000oz_Dec21,O,"5,000","127,000.00",USD,"127,000.00",25.40
4,USD,OC,"-127,000","-127,000.00",USD,"-127,000.00",25.40


We should see no warnings.

In [22]:
display(executed_holdings.warnings)

[]

## The End